In [11]:
import praw
import pandas as pd
from datetime import datetime, timedelta
import time
from transformers import pipeline

reddit = praw.Reddit(
    client_id="k4C4H8PUXa0FGpBrdbZpQg",
    client_secret="1Hv_M0wWh4LLYfpibEA5LGEm5lwH6Q",
    user_agent="search_treasurer"
)

In [8]:

search_term = "ffie"  # Replace with your desired search term
subreddit = reddit.subreddit("FFIE")

data = []
three_days_ago = datetime.utcnow() - timedelta(days=3)
post_count = 0

for submission in subreddit.search(search_term, limit=None):
    if submission.created_utc >= three_days_ago.timestamp():
        author = submission.author.name if submission.author else "[deleted]"
        
        # Add main post to data
        data.append({
            "Content": submission.selftext,
            "Author": author,
            "Subreddit": submission.subreddit.display_name,
            "Score": submission.score,
            "NumComments": submission.num_comments,
            "Created": datetime.fromtimestamp(submission.created_utc),
            "URL": submission.url,
            "Type": "main post"
        })

        submission.comments.replace_more(limit=None)  # Load all comments
        for comment in submission.comments.list():
            comment_author = comment.author.name if comment.author else "[deleted]"

            # Add comment to data
            data.append({
                "Content": comment.body,
                "Author": comment_author,
                "Subreddit": submission.subreddit.display_name,
                "Score": comment.score,
                "NumComments": 0, # Comments don't have nested comments
                "Created": datetime.fromtimestamp(comment.created_utc),
                "URL": comment.permalink,
                "Type": "comment"
            })

        post_count += 1
        if post_count % 100 == 0:
            print(f"Processed {post_count} posts...")

        if post_count >= 10000:  
            break

    time.sleep(2)  # Sleep for 2 seconds between requests (adjust as needed)

df = pd.DataFrame(data)
print(df.head())


                                             Content                Author  \
0  https://preview.redd.it/wqq4lz6mddcd1.png?widt...           dumbmoney99   
1  RedWarFour, would u mind stfu. You have a prob...         victorconan90   
2                                        Helllll yea           Happymeal85   
3                                          Good news  Master_Flounder_9826   
4  The Senior Director role is interesting so it ...       Tall_Walrus6481   

  Subreddit  Score  NumComments             Created  \
0      FFIE    175           42 2024-07-13 16:24:21   
1      FFIE     13            0 2024-07-13 20:07:17   
2      FFIE      9            0 2024-07-13 16:56:11   
3      FFIE      9            0 2024-07-13 18:10:49   
4      FFIE      5            0 2024-07-13 19:23:47   

                                                 URL       Type  
0  https://www.reddit.com/r/FFIE/comments/1e2nvhj...  main post  
1  /r/FFIE/comments/1e2nvhj/ffie_is_hiring_more_j...    comment  
2 

In [9]:


# Get unique authors from the DataFrame
unique_authors = df['Author'].unique()

author_info = []
for author_name in unique_authors:
    if author_name != "[deleted]":
        try:
            redditor = reddit.redditor(author_name)
            author_info.append({
                "Author": redditor.name,
                "Link Karma": redditor.link_karma,
                "Comment Karma": redditor.comment_karma,
                "Is Employee": redditor.is_employee,
                "Is Mod": redditor.is_mod,
                "Account Created": datetime.fromtimestamp(redditor.created_utc)
            })
        except praw.exceptions.RedditAPIException:
            # Handle cases where the redditor might not exist or be inaccessible
            print(f"Could not fetch information for user: {author_name}")
            author_info.append({
                "Author": author_name,
                "Link Karma": None,
                "Comment Karma": None,
                "Is Employee": None,
                "Is Mod": None,
                "Account Created": None
            })
    else:
        author_info.append({
            "Author": author_name,
            "Link Karma": None,
            "Comment Karma": None,
            "Is Employee": None,
            "Is Mod": None,
            "Account Created": None
        })
    time.sleep(1)  # Sleep for 1 second between requests

# Create a new DataFrame with author information
author_df = pd.DataFrame(author_info)
print(author_df.head())


                 Author  Link Karma  Comment Karma  Is Employee  Is Mod  \
0           dumbmoney99       10024           5458        False    True   
1         victorconan90          73            112        False   False   
2           Happymeal85         551           1338        False   False   
3  Master_Flounder_9826           1            742        False   False   
4       Tall_Walrus6481         168           1925        False   False   

      Account Created  
0 2020-09-13 17:22:24  
1 2016-09-19 10:34:06  
2 2021-02-08 17:57:42  
3 2022-03-30 17:39:13  
4 2024-05-17 09:46:29  


In [10]:
df.head()

,Content,Author,Subreddit,Score,NumComments,Created,URL,Type
0,https://preview.redd.it/wqq4lz6mddcd1.png?widt...,dumbmoney99,FFIE,175,42,2024-07-13 16:24:21,https://www.reddit.com/r/FFIE/comments/1e2nvhj...,main post
1,"RedWarFour, would u mind stfu. You have a prob...",victorconan90,FFIE,13,0,2024-07-13 20:07:17,/r/FFIE/comments/1e2nvhj/ffie_is_hiring_more_j...,comment
2,Helllll yea,Happymeal85,FFIE,9,0,2024-07-13 16:56:11,/r/FFIE/comments/1e2nvhj/ffie_is_hiring_more_j...,comment
3,Good news,Master_Flounder_9826,FFIE,9,0,2024-07-13 18:10:49,/r/FFIE/comments/1e2nvhj/ffie_is_hiring_more_j...,comment
4,The Senior Director role is interesting so it ...,Tall_Walrus6481,FFIE,5,0,2024-07-13 19:23:47,/r/FFIE/comments/1e2nvhj/ffie_is_hiring_more_j...,comment


In [13]:

# Load sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english") 

# Function to get sentiment and confidence from text with truncation
def get_sentiment(text, max_length=512): 
    result = sentiment_pipeline(text[:max_length])[0]
    return result['label'], result['score']

# Apply sentiment analysis with truncation to the content column
df[['sentiment', 'sentiment_confidence_score']] = df['Content'].apply(lambda x: pd.Series(get_sentiment(x)))

print(df.head())  # Display the updated DataFrame with sentiment and confidence columns


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

w:\Personal Projects\Nvidia_Gen_AI_notebooks\llmvenv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\golla\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

                                             Content                Author  \
0  https://preview.redd.it/wqq4lz6mddcd1.png?widt...           dumbmoney99   
1  RedWarFour, would u mind stfu. You have a prob...         victorconan90   
2                                        Helllll yea           Happymeal85   
3                                          Good news  Master_Flounder_9826   
4  The Senior Director role is interesting so it ...       Tall_Walrus6481   

  Subreddit  Score  NumComments             Created  \
0      FFIE    175           42 2024-07-13 16:24:21   
1      FFIE     13            0 2024-07-13 20:07:17   
2      FFIE      9            0 2024-07-13 16:56:11   
3      FFIE      9            0 2024-07-13 18:10:49   
4      FFIE      5            0 2024-07-13 19:23:47   

                                                 URL       Type sentiment  \
0  https://www.reddit.com/r/FFIE/comments/1e2nvhj...  main post  NEGATIVE   
1  /r/FFIE/comments/1e2nvhj/ffie_is_hiring_mor

In [15]:
len(df)

148

In [14]:
df['sentiment'].value_counts()

sentiment
NEGATIVE    95
POSITIVE    53
Name: count, dtype: int64

In [19]:
pv1 = pd.pivot_table(df, index='Subreddit', columns='sentiment', values='Content', aggfunc='count', fill_value=0)
pv1

sentiment,NEGATIVE,POSITIVE
Subreddit,,
FFIE,95,53


In [ ]:
import pandas as pd

# Assuming you have df (post/comment data) and author_df (author information) from previous code

# 1. Calculate Unique User Attention per Main Post
main_post_df = df[df['Type'] == 'main post'].copy()
unique_user_counts = main_post_df.groupby('Title')['Author'].nunique()
unique_user_counts.name = 'UniqueUserAttention'

# 2. Merge with Content DataFrame (Main Posts Only)
merged_df = pd.merge(main_post_df, unique_user_counts, on='Title', how='left')

# 3. Merge with Author DataFrame
mega_df = pd.merge(merged_df, author_df, on='Author', how='left')

print(mega_df.head())
